In [ ]:
from confluent_kafka import Producer
import json
import time
import csv
import json
import os, shutil

In [ ]:
def find_rename_csv_file(directory):
    # Check if the directory exists
    if not os.path.exists(directory):
        print(f"Directory '{directory}' does not exist.")
        return None

    # Traverse through the directory and its subdirectories
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file is a CSV file
            if file.endswith('.csv'):
                new_name = os.path.join(root,file.split(".")[0] +"__"+ root.split("\\")[-1] +".csv")
                os.rename(os.path.join(root, file), new_name)
                return new_name

    print(f"No CSV files found in '{directory}' or its subdirectories.")
    return None

In [ ]:
def move_file(source_file_path, destination_folder):
    # Check if the source file exists
    if not os.path.exists(source_file_path):
        print(f"Source file '{source_file_path}' does not exist.")
        return
    
    # Check if the destination folder exists, if not, create it
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    
    # Extract the filename from the source file path
    file_name = os.path.basename(source_file_path)
    
    # Move the file to the destination folder
    destination_path = os.path.join(destination_folder, file_name)
    shutil.move(source_file_path, destination_path)
    print(f"Moved '{file_name}' to '{destination_folder}'.")
    return destination_path

In [ ]:
def csv_to_json(csv_file, json_file):
    # Open the CSV file and load data into a Python dictionary
    with open(csv_file, 'r') as file:
        csv_data = csv.DictReader(file)
        data = [row for row in csv_data]

    # Write the Python dictionary into a JSON file
    with open(json_file, 'w') as jsonfile:
        jsonfile.write(json.dumps(data))

    return json.dumps(data)

In [ ]:
class KafkaProducer:
    def __init__(self):
        self.topic = "xxxx"
        self.conf = {'bootstrap.servers': 'xxxx',
                     'security.protocol': 'xxxx',
                     'sasl.mechanism': 'xxxx',
                     'sasl.username': 'xxxx',
                     'sasl.password': 'xxxx',
                     'client.id': "xxxx"}
        

    def delivery_callback(self, err, msg):
        if err:
            print('ERROR: Message failed delivery: {}'.format(err))
        else:
            key = msg.key().decode('utf-8')

    def produce_invoices(self, producer,csv_file_path):

        timestamp_ms = int(time.time() * 1000)
        json_data=csv_to_json(csv_file_path,"myjson.json")
        producer.produce(self.topic, key=str(1), timestamp=timestamp_ms ,value=json_data, callback=self.delivery_callback)
        print(f"File: {csv_file_path} --> imported")
        

    def start(self, csv_file_path):
        kafka_producer = Producer(self.conf)
        self.produce_invoices(kafka_producer,csv_file_path)
        kafka_producer.flush()

In [ ]:
if __name__ == "__main__":

    directory_path = 'C:\PerfLogs\Admin\MyLogger'
    destination_folder = 'C:\\PerfLogs\\Admin\\tmp'
    destination_folder_working = 'C:\\PerfLogs\\Admin\\working'
    invoice_producer = KafkaProducer()
    
    while(True):
        csv_file_path = find_rename_csv_file(directory_path)
        if csv_file_path:
            try:
                wrkpath = move_file(csv_file_path, destination_folder_working)
                invoice_producer.start(wrkpath)
                wrkpath = move_file(wrkpath, destination_folder)
            except:
                print("waiting 1 minute")
                time.sleep(60)
        time.sleep(5)